In [1]:
import pandas as pd
import numpy as np
from pandas import *
from math import *


# Part 1: Read Raw data

In [2]:
df = pd.read_csv('data/all_stocks_5yr.csv')
df['date'] = pd.to_datetime(df['date'])
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [3]:
print(f'Dataframe shape: {df.shape}')

Dataframe shape: (619040, 7)


# Part 2: generate features from time series data <br>

#### Take AAL as an example

In [4]:
AAL = df[df.Name=="AAL"]
print(f'Dataframe shape: {AAL.shape}')
AAL.head(5)

Dataframe shape: (1259, 7)


,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


### 2.1 Generate monthly return/vol  features

In [5]:
AAL_month = AAL.groupby(list(map(lambda x:x.month,AAL.date))).nth(0).reset_index(drop=True) ### nth means get first row of the grouped data
AAL_month.head(5)

,date,open,high,low,close,volume,Name
0,2014-01-02,25.07,25.82,25.06,25.36,8998943,AAL
1,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
2,2013-03-01,13.37,13.95,13.32,13.61,7376800,AAL
3,2013-04-01,17.02,17.13,16.54,16.67,5222300,AAL
4,2013-05-01,16.91,17.17,16.60,16.60,4943600,AAL


In [6]:
### Generate monthly return
AAL_month['return'] = AAL_month.close.shift(1)/AAL_month.close - 1
AAL_month.head()

,date,open,high,low,close,volume,Name,return
0,2014-01-02,25.07,25.82,25.06,25.36,8998943,AAL,NaN
1,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,0.719322
2,2013-03-01,13.37,13.95,13.32,13.61,7376800,AAL,0.083762
3,2013-04-01,17.02,17.13,16.54,16.67,5222300,AAL,-0.183563
4,2013-05-01,16.91,17.17,16.60,16.60,4943600,AAL,0.004217


In [7]:
AAL_month['vol_change'] = AAL_month.volume.shift(1)/AAL_month.volume - 1
AAL_month.head()

,date,open,high,low,close,volume,Name,return,vol_change
0,2014-01-02,25.07,25.82,25.06,25.36,8998943,AAL,NaN,NaN
1,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,0.719322,0.070347
2,2013-03-01,13.37,13.95,13.32,13.61,7376800,AAL,0.083762,0.139722
3,2013-04-01,17.02,17.13,16.54,16.67,5222300,AAL,-0.183563,0.412558
4,2013-05-01,16.91,17.17,16.60,16.60,4943600,AAL,0.004217,0.056376


In [8]:
def generate_features(df):
    date_month = list(map(lambda x:x.month,df.date))
    df_monthly = df.groupby(date_month).nth(0).reset_index(drop=True)
    df_monthly['return'] = df_monthly.close.shift(1)/df_monthly.close - 1
    df_monthly['vol_change'] = df_monthly.volume.shift(1)/df_monthly.volume - 1
    return df_monthly[1:] ### [1:] means jump the first row

In [9]:
test_AAL = generate_features(AAL)
test_AAL.head()

,date,open,high,low,close,volume,Name,return,vol_change
1,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,0.719322,0.070347
2,2013-03-01,13.37,13.95,13.32,13.61,7376800,AAL,0.083762,0.139722
3,2013-04-01,17.02,17.13,16.54,16.67,5222300,AAL,-0.183563,0.412558
4,2013-05-01,16.91,17.17,16.60,16.60,4943600,AAL,0.004217,0.056376
5,2013-06-03,17.54,17.90,17.40,17.73,5776800,AAL,-0.063734,-0.144232


###  2.2 Generate technical analysis features 

- ROC : rate of change
- MOM : momentum
- MA : Moving Average
- MACD : moving average convergence/divergence


In [24]:
df = pd.read_csv('data/all_stocks_5yr.csv')
df['date'] = pd.to_datetime(df['date'])
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


#### 2.2.1 ROC:

In [11]:
n = 10

In [12]:
M = df['close'].diff(n - 1)
N = df['close'].shift(n - 1)
ROC = Series(M / N, name = 'ROC_' + str(n))
df.join(ROC).iloc[10:15]

,date,open,high,low,close,volume,Name,ROC_10
10,2013-02-25,13.60,13.76,13.00,13.02,7186400,AAL,-0.099585
11,2013-02-26,13.14,13.42,12.70,13.26,9419000,AAL,-0.070778
12,2013-02-27,13.28,13.62,13.18,13.41,7390500,AAL,-0.085266
13,2013-02-28,13.49,13.63,13.39,13.43,6143600,AAL,-0.040029
14,2013-03-01,13.37,13.95,13.32,13.61,7376800,AAL,-0.061379


#### 2.2.2 MOM: momentum

In [13]:
n = 3

In [14]:
M = Series(df['close'].diff(n), name = 'Momentum_' + str(n))
df.join(M).iloc[10:15]

,date,open,high,low,close,volume,Name,Momentum_3
10,2013-02-25,13.60,13.76,13.00,13.02,7186400,AAL,-0.31
11,2013-02-26,13.14,13.42,12.70,13.26,9419000,AAL,-0.11
12,2013-02-27,13.28,13.62,13.18,13.41,7390500,AAL,-0.16
13,2013-02-28,13.49,13.63,13.39,13.43,6143600,AAL,0.41
14,2013-03-01,13.37,13.95,13.32,13.61,7376800,AAL,0.35


#### 2.2.3 Moving Average

In [15]:
n = 5

In [16]:
#Moving Average
MA = Series(df.close.rolling(n).mean(), name = 'MA_' + str(n))

#Exponential Moving Average
EMA = df['close'].ewm(span = n, min_periods = n - 1).mean()

#### 2.2.4 MACD

In [17]:
"""Param of MACD"""
n_fast = 12
n_slow = 26

In [45]:
EMAfast = pd.DataFrame(df['close']).ewm(span = n_fast, min_periods = n_fast - 1).mean()
EMAslow = pd.DataFrame(df['close']).ewm(span = n_slow, min_periods = n_slow - 1).mean()

MACD = EMAfast - EMAslow
MACD.columns = ['MACD_' + str(n_fast) + '_' + str(n_slow)]

In [46]:
MACDsign = MACD.ewm(span = 9, min_periods = 8).mean()
MACDsign.columns = ['MACDsign_' + str(n_fast) + '_' + str(n_slow)]

In [47]:
MACDdiff = pd.DataFrame(MACD.values.reshape(-1) - MACDsign.values.reshape(-1),index=MACDsign.index)
MACDdiff.columns = ['MACDdiff_' + str(n_fast) + '_' + str(n_slow)]

In [65]:
MACD.values.shape

(619040, 1)

# Part 3: Package 

In [ ]:
def generate_monthly_features(self,df):
    df_featured = df.groupby(["Name"]).apply(self.helper_monthly_features)
    return df_featured

In [230]:
tmp = {"n_fast":12,"n_slow":26}

In [22]:
import datetime

In [23]:
datetime.date(month=2,year=2014,day=1)

datetime.date(2014, 2, 1)

In [32]:
date_month = list(map(lambda x:datetime.date(month=x.month,year=x.year,day=1),df.date))

df.groupby(date_month).nth(0)

In [105]:
class processor:
    def __init__(self,config=None,**kwargs):
        ### config is a dictionary contains all 
        self.config = config if config else {"MACD":{"n_fast":12,"n_slow":26},"ROC":[{"n":5}],"MOM":[{"n":3}],"MA":[{"n":5}],"EMA":[{"n":3}]}
        
    def process(self,df):
        for i in self.config:
            if i == "MACD":
                df = self.MACD(df,**self.config[i])
            if i == "ROC":
                for tmpConf in self.config[i]:
                    df = self.ROC(df,**tmpConf)
            if i == "MA":
                for tmpConf in self.config[i]:
                    df = self.MA(df,**tmpConf)
            if i == "EMA":
                for tmpConf in self.config[i]:
                    df = self.EMA(df,**tmpConf)
            if i == "MOM":
                for tmpConf in self.config[i]:
                    df = self.MOM(df,**tmpConf)

        return df

    def MACD(self,df, n_fast, n_slow,**kwargs):
        EMAfast = pd.DataFrame(df['close']).ewm(span = n_fast, min_periods = n_fast - 1).mean()
        EMAslow = pd.DataFrame(df['close']).ewm(span = n_slow, min_periods = n_slow - 1).mean()

        MACD = EMAfast - EMAslow
        MACD.columns = ['MACD_' + str(n_fast) + '_' + str(n_slow)]

        MACDsign = MACD.ewm(span = 9, min_periods = 8).mean()
        MACDsign.columns = ['MACDsign_' + str(n_fast) + '_' + str(n_slow)]

        MACDdiff = pd.DataFrame(MACD.values.reshape(-1) - MACDsign.values.reshape(-1),index=MACDsign.index)
        MACDdiff.columns = ['MACDdiff_' + str(n_fast) + '_' + str(n_slow)]
        
        df = df.join(MACD).join(MACDsign).join(MACDdiff)
        return df
    
    #Moving Average
    def MA(self,df, n):
        MA = Series(df.close.rolling(n).mean(), name = 'MA_' + str(n))
        df = df.join(MA)
        return df

    #Exponential Moving Average
    def EMA(self,df, n):
        EMA = pd.DataFrame(df['close'].ewm(span = n, min_periods = n - 1).mean())
        EMA.columns= ['EMA_' + str(n)]
        df = df.join(EMA)
        return df
    #Rate of Change
    def ROC(self,df, n,**kwargs):
        M = df['close'].diff(n - 1)
        N = df['close'].shift(n - 1)
        ROC = Series(M / N, name = 'ROC_' + str(n))
        df = df.join(ROC)
        return df
    
    #Momentum
    def MOM(self,df, n,**kwargs):
        M = Series(df['close'].diff(n), name = 'Momentum_' + str(n))
        df = df.join(M)
        return df

In [106]:
myProcessor = processor()

In [109]:
df_featured = df.groupby(df.Name).apply(lambda x:myProcessor.process(x)).reset_index(drop=True)

In [110]:
df_featured[df_featured.Name=='AAL']

,date,open,high,low,close,volume,Name,MACD_12_26,MACDsign_12_26,MACDdiff_12_26,ROC_5,Momentum_3,MA_5,EMA_3
0,2013-02-08,15.07,15.1200,14.6300,14.75,8407500,AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-02-11,14.89,15.0100,14.2600,14.46,8882000,AAL,NaN,NaN,NaN,NaN,NaN,NaN,14.556667
2,2013-02-12,14.45,14.5100,14.1000,14.27,8126000,AAL,NaN,NaN,NaN,NaN,NaN,NaN,14.392857
3,2013-02-13,14.30,14.9400,14.2500,14.66,10259500,AAL,NaN,NaN,NaN,NaN,-0.09,NaN,14.535333
4,2013-02-14,14.94,14.9600,13.1600,13.99,31879900,AAL,NaN,NaN,NaN,-0.051525,-0.47,14.426,14.253871
5,2013-02-15,13.93,14.6100,13.9300,14.50,15628000,AAL,NaN,NaN,NaN,0.002766,0.23,14.376,14.378889
6,2013-02-19,14.33,14.5600,14.0800,14.26,11354400,AAL,NaN,NaN,NaN,-0.000701,-0.40,14.336,14.318976
7,2013-02-20,14.17,14.2600,13.1500,13.33,14725200,AAL,NaN,NaN,NaN,-0.090723,-0.66,14.148,13.822549
8,2013-02-21,13.62,13.9500,12.9000,13.37,11922100,AAL,NaN,NaN,NaN,-0.044317,-1.13,13.890,13.595832
9,2013-02-22,13.57,13.6000,13.2100,13.57,6071400,AAL,NaN,NaN,NaN,-0.064138,-0.69,13.806,13.582903


In [111]:
df_featured.to_csv("data/df_featured.csv",index=False)